# Operações em DataFrames

Hoje veremos algumos operações fundamentais em DataFrames:

1. **Renomeando colunas/linhas**;
1. **Reordenando linhas**;
1. **Removendo colunas/linhas**;
1. **Filtrando dados**;
1. **Criação de colunas condicionais**.

Vamos começar importando as bibliotecas Numpy e Pandas (seguindo a conveção de importação `np` e `pd`) e utilizar a função `pd.read_csv()`para carregar um arquivo texto com dados sobre carros.

Não se preocupe com a utilização da função `pd.read_csv()` no momento - na próxima aula aprenderemos a carregar diferentes arquivos de dados, incluindo arquivos `.csv`!

In [ ]:
import pandas as pd
import numpy as np
tb_veic = pd.read_csv('data/dados_veiculos.csv')

Podemos usar o método `.info()` para ver quais informações a tabela contém:

In [ ]:
tb_veic.info()

Como podemos ver abaixo, a tabela contém 15 colunas, sendo 9 numéricas e 6 de objetos. Além disso todas as linhas parecem ter todas as variáveis preenchidas. Podemos utilizar o método `.head()` para ver as primeiras linhas dessa tabela:

In [ ]:
tb_veic.head()

Qual método podemos utilizar para ter uma descrição rápida sobre as colunas numéricas dessa tabela?

In [ ]:
# SEU CÓDIGO AQUI!

## Manipulando Colunas

DataFrames são tabelas - portanto podemos realizar operações sobre as **colunas** ou sobre as **linhas**. Vamos começar vendo algumas operações básicas que podemos realizar sobre as colunas de um DataFrame.

### O atributo `.columns`

Como vimos na aula passada, todo DataFrame contém atributos que nos permite acessar seus índices. O atributo para acessarmos o índice de colunas é o `.columns`. Vamos utilizar esse atributo para resolver um problema básico: o método `.describe()` só retorna resultados para colunas não numéricas quando o DataFrame não tem nenhuma colunas numérica.

No entanto, muitas vezes queremos ver o resumo fornecido pelo método para as colunas de `object` (que, por via de regra, são `strings`). Como podemos fazer isso?

In [ ]:
# Vamos resolver juntos!

### Renomeando colunas
Temos duas formas possíveis de renomear as colunas:

- Como já vimos `.columns` é um **attribute** dos DataFrame que contém um iterável com os nomes das colunas:
    - Podemos substituir esse atributo por outro iterável de mesmo comprimento;
    - Como substituímos o atributo inteiro, precisamos especificar o nome de todas as colunas (mesmo que elas não mudem de nome).
    
- `.rename()` é um **método** de um DataFrame, através do qual podemos renomear colunas de forma seletiva:
    - Este método utiliza um dicionário com o formato `{'nome_antigo':'nome_novo'}` para renomear as colunas; 
    - O método `.rename()` serve tanto para renomear colunas quanto linhas. Portanto precisamos utilizar o argumento `axis = 1` para renomear as colunas;

#### Através do atributo `.columns`
Primeiro vamos aprender a renomear colunas através do atributo `.columns`. Além disso vamos ver situações práticas onde devemos utilizar essa forma.

In [ ]:
print(tb_veic.columns)

Utilizando um list comprehesion podemos criar uma nova lista de nomes de colunas:

In [ ]:
old_names = tb_veic.columns
print(old_names)
new_names = [f'C_{i}' for i in range(len(old_names))]
print(new_names)

Agora, podemos substituir o atributo `.columns` e utilizar o método `.info()` para verificar que nossas colunas foram renomeadas:

In [ ]:
tb_veic.columns = new_names
tb_veic.info()

Podemos utilizar os nomes antigos guardados na variável `old_names` para voltar a tabela ao normal:

In [ ]:
tb_veic.columns = old_names
tb_veic.info()

O exemplo acima mostra como podemos utilizar o atributo `.columns` para renomear colunas. No entanto, ele não é muito prático. Vamos ver um caso real onde utilizamos essa técnica para limpar o nome das colunas de uma tabela:

In [ ]:
import re

pattern = r'[^a-zA-Z0-9]'
print(re.findall(pattern, tb_veic.columns[0]))

O padrão acima busca todos os carácteres que **NÃO** são alfa-numéricos. Podemos utilizar a função `re.sub()` para *limpar* os nomes das colunas, substituindo espaços e barras por `_`:

In [ ]:
print([re.sub(pattern, '_', column.lower()) for column in tb_veic.columns])

In [ ]:
tb_veic.columns = [re.sub(pattern, '_', column.lower()) for column in tb_veic.columns]
tb_veic.info()

In [ ]:
tb_veic.columns = old_names
tb_veic.info()

### Utilizando o método `.rename()`

Um problema do método acima é que se quisermos renomear apenas uma coluna teremos que criar uma lista com todos os nomes originais exceto o da coluna a ser renomeada. Neste caso é melhor utilizarmos o método `.rename()`: ele nos permite renomear uma (ou mais) colunas a partir de um dicionário `{'nome_antigo' : 'nome_novo'}`:

In [ ]:
dict_nomes = dict()
dict_nomes['Year'] = 'model_year'
print(dict_nomes)

In [ ]:
tb_veic = tb_veic.rename(dict_nomes, axis = 1)
tb_veic.info()

In [ ]:
tb_veic.columns = old_names
tb_veic.info()

O método `.rename()` não altera o objeto original!

In [ ]:
tb_veic.rename({'Year' : 'model_year'}, axis = 1)
tb_veic.info()

Se quisermos que o método substitua diretamente o nome das colunas (sem precisarmos sobrescrever a variável que contém o DataFrame) precisamos utilizar o argumento `inplace = True`:

In [ ]:
tb_veic.rename({'Year' : 'model_year'}, axis = 1, inplace = True)
tb_veic.info()

Podemos realizar a mesma tarefa do exemplo prático acima através do método `.rename()`. Para tanto, vamos utilizar um `dict comprehension`!

In [ ]:
pattern = r'[^a-zA-Z0-9]'
dict_rename = {column : re.sub(pattern, '_', column).lower() for column in tb_veic.columns}
print(dict_rename)

In [ ]:
tb_veic = tb_veic.rename(dict_rename, axis = 1) #tb_veic.rename(dict_rename, axis = 1, inplace = True)
print(tb_veic.columns)

Utilizando o método `.replace()` temos duas formas de renomear as colunas de nossos dados
1. sobrescrevendo a variável que contém nosso `DataFrame`: 

    `data = data.rename(columns={'Make':'Manufacturer', 'Year':'ANO'})`

2. Usando o argumento `inplace =  True`:

    `data.rename(columns={'Make':'Manufacturer', 'Year':'ANO'}, inplace=True)`
    
O parâmetro 'inplace' será deprecado e seu uso é considerado má prática.

### Reordenando colunas em um DataFrame

Reordenar colunas em um DataFrame é simples: basta lembrarmos que podemos passar uma lista de nomes de colunas como índice para montar um novo DataFrame com as colunas na ordem dos elementos da lista:

In [ ]:
print(tb_veic.columns)

In [ ]:
tb_veic[['model', 'make']]

Vamos utilizar o método `.sort()` das listas para criar uma lista de colunas em ordem alfabética:

In [ ]:
lista_colunas = list(tb_veic.columns)
lista_colunas.sort()
print(lista_colunas)

In [ ]:
tb_veic = tb_veic[lista_colunas]
tb_veic.head()

### Removendo/Mantendo colunas

Se quisermos remover uma (ou mais) coluna de um DataFrame podemos faze-lo de duas formas:

- Utilizando a indexação para selecionar todas as colunas que queremos em nosso novo DataFrame;
- Utilizando o método `.drop()` para remover algumas colunas específicas.

Quando queremos remover muitas colunas, pode valer a penas especificar quais colunas queremos manter (primeira forma). Caso contrário, podemos utilizar o método `.drop()`:

In [ ]:
colunas_manter = ['city_mpg', 'combined_mpg']
tb_veic_sub = tb_veic[colunas_manter]
tb_veic_sub.info()

In [ ]:
colunas_num = [coluna for coluna in tb_veic.columns if tb_veic[coluna].dtype == int or tb_veic[coluna].dtype == float]
print(colunas_num)

In [ ]:
tb_veic_num = tb_veic[colunas_num]
tb_veic_num.info()

Vamos voltar ao exercicio do começo da aula:

In [ ]:
colunas_str = [coluna for coluna in tb_veic.columns if tb_veic[coluna].dtype == object]
tb_veic[colunas_str].describe()

As técnicas acima nos permite selecionar colunas de forma simples a partir de condições específicas. No entanto, muitas vezes queremos remover apenas uma ou duas colunas de uma tabela (por exemplo, colunas que não tenham informações corretas). Para isso, podemos utilizar o método `.drop()`:

In [ ]:
tb_veic_sm = tb_veic.drop('make', axis = 1)
tb_veic_sm.info()

Esse método nos permite remover mais de uma coluna (utilizando um iterável) e aceita o argumento `inplace`:

In [ ]:
tb_veic_sm.drop(['transmission', 'model_year'], axis = 1, inplace = True)
tb_veic_sm.info()

## Manipulando Linhas

In [ ]:
tb_veic = tb_veic.sort_values('model_year')
tb_veic

In [ ]:
tb_veic = tb_veic.sort_values('model_year', ascending = False)
tb_veic

In [ ]:
tb_veic = tb_veic.sort_values(['model_year', 'engine_displacement'], ascending = [False, True])
tb_veic

In [ ]:
tb_veic = tb_veic.sort_values(['model_year', 'engine_displacement', 'make'], ascending = False)
tb_veic

### Aplicando Filtros

Como vimos nas últimas aulas, o conceito de filtro é fundamental na programação para análise de dados. Vamos aplicar o que aprendemos sobre um conjunto de dados real.

Na biblioteca Pandas temos duas maneira de realizar filtros:

- Conceito de máscara;
- O método `.query()`.

Começaremos pelas máscaras.

In [ ]:
cyl_6 = tb_veic['cylinders'] == 6
print(cyl_6)
sum(cyl_6)

In [ ]:
tb_veic_cyl6 = tb_veic[tb_veic['cylinders'] == 6]
tb_veic_cyl6

In [ ]:
tb_veic_cyl6.describe()

#### Combinando condições

Podemos utilizar os operadores `&` (análogo ao `and`) e `|` (análogo ao `or`) para combinar condições de forma complexa.

Vamos começar com um problema simples: criar um DataFrame com carros da `Ford` que tenham 6 cilindros.

In [ ]:
print(tb_veic.columns)

In [ ]:
tb_veic['make'].value_counts()

In [ ]:
print(tb_veic['make'] == 'Ford')

In [ ]:
sum(tb_veic['make'] == 'Ford')

In [ ]:
sum((tb_veic['make'] == 'Ford') & (tb_veic['cylinders'] == 6))

In [ ]:
mask_ford_6 = (tb_veic['make'] == 'Ford') & (tb_veic['cylinders'] == 6)
tb_ford6 = tb_veic[mask_ford_6]
tb_ford6.describe()

E se quisessemos construir um DataFrame com todos os carros da Ford de 6 cilindros e todos os carros da Chevrolet de 8 cilindros?

In [ ]:
# SEU CÓDIGO AQUI

### Criando colunas condicionais

Além de ser extremamente útil para criar sub-conjuntos de dados, as máscaras também são utilizadas para criar **colunas condicionais**: colunas cujo valor é determinado a partir de condicionais sobre os valores de outras colunas.

#### Utilizando `.loc`
Vamos começar com um exemplo simples: ao invés de filtrar nossos dados, vamos criar uma coluna binária indicando se um carro é da marca Ford. A primeira forma de fazermos isto é através do atributo `.loc`.

In [ ]:
mask_ford = tb_veic['make'] == 'Ford'
print(mask_ford)

Vamos relembrar como a indexação utilizando `.loc` funciona:

``` python
data.loc[row_name, column_name]
```

Podemos passar nossa máscara como índice das linhas, no `row_name`, e podemos criar nossa coluna passando um `column_name` que ainda não existe em nosso DataFrame!

In [ ]:
tb_veic.loc[mask_ford, 'e_ford'] = 1
tb_veic.loc[~mask_ford, 'e_ford'] = 0

In [ ]:
tb_veic['e_ford'].describe()

In [ ]:
tb_veic[~mask_ford].head()

Vamos utilizar essa mesma construção para criar uma classificação de eficiência dos carros, através da coluna `city_mpg`:

- Carros que fazem **menos que 15 Milhas por Galão** serão categoria **C**;
- Carros que fazem **entre 15 e 20 Milhas por Galão** serão categoria **B**;
- Carros que fazem **mais que 20 Milhas por Galão** serão categoria **A**.

Vamos guardar o resultado dessa classificação na coluna `eff_city`.

In [ ]:
mask_C = tb_veic['city_mpg']  < 15
mask_B = (tb_veic['city_mpg']  >= 15) & (tb_veic['city_mpg']  < 20)
mask_A = tb_veic['city_mpg']  >= 20

In [ ]:
tb_veic.loc[mask_C, 'eff_city'] = 'C'
tb_veic.loc[mask_B, 'eff_city'] = 'B'
tb_veic.loc[mask_A, 'eff_city'] = 'A'

In [ ]:
tb_veic['eff_city'].value_counts()

Poderíamos ter criado esta coluna de forma mais abreviada, sem inicializar variáveis para cada máscara:

In [ ]:
tb_veic.loc[tb_veic['city_mpg']  < 15, 'eff_city'] = 'C'
tb_veic.loc[(tb_veic['city_mpg']  >= 15) & (tb_veic['city_mpg']  < 20), 'eff_city'] = 'B'
tb_veic.loc[tb_veic['city_mpg']  >= 20, 'eff_city'] = 'A'

In [ ]:
tb_veic.columns

As condições acima são **completas**, ou seja, todas as linhas de nossa tabela se enquadrarão em uma das três categorias. O que acontece quando inicializamos uma **coluna condicional** com uma máscara incompleta?

Vamos entender esse comportamento criando a coluna `eff_high`, construída a partir da coluna `highway_mpg` utilizando as condições:

- Carros que fazem **menos que 20 Milhas por Galão** serão categoria **C**;
- Carros que fazem **entre 20 e 30 Milhas por Galão** serão categoria **B**.

(O que está faltando para a condição ser completa?)

In [ ]:
mask_C = tb_veic['highway_mpg']  < 20
mask_B = (tb_veic['highway_mpg']  >= 20) & (tb_veic['highway_mpg']  < 30)

In [ ]:
tb_veic.loc[mask_C, 'eff_high'] = 'C'
tb_veic.loc[mask_B, 'eff_high'] = 'B'
tb_veic.info()

In [ ]:
tb_veic.loc[~mask_C & ~mask_B]

#### Utilizando `np.where()`

Além de utilizar a indexação através do `.loc`, podemos utilizar a função `np.where()` do NumPy para criar colunas condicionais. A sintaxe desta função é:

`np.where(Máscara, Valor quando Máscara é Verdadeira, Valor quando False)`

O `np.where()` tem uma sintáxe muito parecida com o `if/else` (para quem já usou Excel, é o próprio IF de planilhas!). Vamos começar criando uma coluna binária simples: `cyl_6`. O valor dessa coluna será `1` quando o carro tiver 6 cilindros e `0` em todos os outros casos.

In [ ]:
tb_veic['cyl_6'] = np.where(tb_veic['cylinders'] == 6, 1, 0)
tb_veic['cyl_6'].describe()

Agora, vamos construir condições mais complexas.

Primeiro, vamos utilizar duas condicionais em uma máscara para criar a coluna `cyl_6_ford`: uma marcação binária dos carros da Ford com 6 cilindros.

In [ ]:
tb_veic['cyl_6_ford'] = np.where((tb_veic['make'] == 'Ford') & (tb_veic['cylinders'] == 6),
                                 1,
                                 0)
print(tb_veic['cyl_6_ford'].describe())

Agora vamos encadear condicionais para recriar a regra de eficiência na estrada, `eff_high`, a partir da coluna `highway_mpg`:

- Carros que fazem **menos que 20 Milhas por Galão** serão categoria **C**;
- Carros que fazem **entre 20 e 30 Milhas por Galão** serão categoria **B**;
- Carros que fazem **mais que 30 Milhas por Galão** serão categoria **B**;

In [ ]:
tb_veic['eff_high'] = np.where(tb_veic['highway_mpg'] < 20, 'C', np.where(tb_veic['highway_mpg'] < 30, 'B', 'A'))
tb_veic['eff_high'].value_counts()